## Assigments 1.1

This first part of the assignments is based on the iris dataset (https://en.wikipedia.org/wiki/Iris_flower_data_set)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

# importing the data from sklearn
from sklearn.datasets import load_iris
iris_dataset = load_iris()

from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score

# extracting the relevant information

data = pd.DataFrame(data=iris_dataset.data, columns=iris_dataset.feature_names)
data_feature_names = data.columns
target = iris_dataset.target
target_names = iris_dataset.target_names

print('There are ' + str(len(data_feature_names)) + ' features, whose names are: \n' + str(data_feature_names))
print('\nThere are ' + str(len(target_names)) + ' classes to predict, whose names are: \n' + str(target_names))
print('\nThere are ' + str(data.shape[0]) + ' observations')

There are 4 features, whose names are: 
Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)'],
      dtype='object')

There are 3 classes to predict, whose names are: 
['setosa' 'versicolor' 'virginica']

There are 150 observations


$\mathbf{Exercise\, 1.}$ Compute the median, mean and standard deviation for each of the 4 features in the iris dataset (don't use numpy or other pre-defined functions, write your own routine).

In [ ]:
def mean(data):
  return data.sum() / len(data)

def variance(data):
  data_mean = mean(data)

  return ((data - data_mean)**2).sum() / (len(data)-1)

def std(data):
  data_var = variance(data)

  assert data_var >= 0
  return (data_var) ** (1/2)

dataframe = pd.DataFrame(columns=data.columns)
dataframe.loc['mean'] = data.apply(lambda data: mean(data))
dataframe.loc['variance'] = data.apply(lambda data: variance(data))
dataframe.loc['std'] = data.apply(lambda data: std(data))
dataframe

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
mean,5.843333,3.057333,3.758000,1.199333
variance,0.685694,0.189979,3.116278,0.581006
std,0.828066,0.435866,1.765298,0.762238


In [ ]:
df = pd.DataFrame(columns=data.columns)
df.loc["mean"] = data.mean()
df.loc["variance"] = data.var()
df.loc["std"] = data.std()

df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
mean,5.843333,3.057333,3.758000,1.199333
variance,0.685694,0.189979,3.116278,0.581006
std,0.828066,0.435866,1.765298,0.762238


In [ ]:
np.allclose(dataframe, df)

True

$\mathbf{Exercise\, 2.}$ For each feature, generate 150 Gaussian distributed samples with the same mean and standard_deviation computed in Exercise 1.

In [ ]:
df_gauss = pd.DataFrame()

for col in dataframe.columns:
  df_gauss[col] = np.random.normal(
      loc=dataframe.loc['mean', col], 
      scale=dataframe.loc['std', col],
      size=150
  )

df_gauss

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,6.470191,3.466698,0.122732,0.162504
1,5.147866,3.227266,4.208309,0.551715
2,6.306156,2.908065,1.749647,0.529184
3,6.439659,3.217054,2.751251,1.303847
4,6.215903,2.220097,2.007907,2.448754
...,...,...,...,...
145,6.788503,3.529598,4.785142,0.590447
146,5.826130,3.384414,3.225354,1.407513
147,4.894114,2.429072,6.474246,2.462463
148,4.523320,2.780251,4.411468,1.600086


$\mathbf{Exercise\, 3.}$ Compute the histogram (with 30 bins) of the distribution of the generate samples and plot it together with the histogram of the original data. The histograms must be plotted on the same plot. Comment on the difference between simulations and real data: for which feature the distributions are most different? What is the reason? 

In [ ]:
# create df for data, with column indicating origin
df_comb = data.copy()
df_comb["origin"] = "data"

# create df gauss with column indicating origin
df_append = df_gauss.copy()
df_append["origin"] = "gauss"

# combine the two dataframes
df_comb = df_comb.append(other=df_append, ignore_index=True)
df_comb = pd.melt(
    frame=df_comb, id_vars="origin", var_name="feature"
)  # get feature name as a column
df_comb

,origin,feature,value
0,data,sepal length (cm),5.100000
1,data,sepal length (cm),4.900000
2,data,sepal length (cm),4.700000
3,data,sepal length (cm),4.600000
4,data,sepal length (cm),5.000000
...,...,...,...
1195,gauss,petal width (cm),0.590447
1196,gauss,petal width (cm),1.407513
1197,gauss,petal width (cm),2.462463
1198,gauss,petal width (cm),1.600086


In [ ]:
col = data.columns[0]

fig = px.histogram(
    data_frame=df_comb,
    barmode="overlay",
    color_discrete_sequence=px.colors.qualitative.G10,
    opacity=0.6,
    nbins=50,
    title="Comparison between original and synthetic gaussian data",
    color="origin",
    facet_col="feature",
)
fig

On the graph we can see that the last 2 features petal length and petal width have the most difference from the original data.

It may depend on the fact that so far petal length had the biggest standard deviation in the features. But if we look only at the standard deviation of a feature the sepal length also had a big std but on the plot we can see that the difference between the original and generated data is not that big.

This can mean that either petal length original data is not of gaussian distribution or it is gaussian but with less representative abilities than the sepal length.

$\mathbf{Exercise\, 4.}$ Estimate and plot the probability density function of the feature $\mbox{petal length (cm)}$.


In [ ]:
import plotly.figure_factory as ff

ff.create_distplot(
    hist_data=[data["petal length (cm)"].values],
    group_labels=["petal length (cm)"],
    bin_size=0.5,
    curve_type='kde'
)


## Assigment 1.2

$\mathbf{Exercise\, 1.}$ Create a linear benchmark, where the dimension of the input data $X$ is 2, and the dimension of the output data $y$ is 1. Choose the amount of noise and sample size.

Evaluate the prediction metrics for the following regression methods:

- Linear regression
- Support Vector Regression with RBF kernel
- Decision Tree Regression


$\mathbf{Exercise\, 2.}$ Create a testing dataset and assess the testing metrics for the models.

In [ ]:
def generate_data(n_samples, n_features, noise_scale):
    X = np.random.uniform(-6, 6, size=(n_samples, n_features))
    y = X.sum(axis=1) + np.random.uniform(-noise_scale, noise_scale, size=n_samples)
    return X, y

def fit_linear_model(X, y):
    return np.mean(X * y[:, np.newaxis], axis=0)

def predict(X, w):
    return X @ w

def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Input data dimension: 2
# Output data dimension: 1
n_samples = 3000
n_features = 2
noise_scale = 5

# Generating data
X, y = generate_data(n_samples, n_features, noise_scale)

# Fitting the linear model on the generated data
w = fit_linear_model(X, y)

# Making predictions on the input data using the fitted model
y_pred = predict(X, w)

# Calculating the mean squared error between the true and predicted outputs
mse = mean_squared_error(y, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 3143.060665756356


In [ ]:
px.scatter_3d(x=X[:, 0].flatten(), y=X[:, 1].flatten(), z=y.flatten()).show()

When we dont add noise (noise_scale == 0) we can see that the graph has a perfect shape of a plane.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2400, 2), (600, 2), (2400,), (600,))

In [ ]:
best_regressor = {}

In [ ]:
def MSE(y_test, y_pred):
  mse = mean_squared_error(y_test, y_pred)
  return mse

def R2(y_test, y_pred):
  r2 = r2_score(y_test, y_pred)
  return r2

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# create regressor
lin_reg = LinearRegression()

# fit it
lin_reg.fit(X_train, y_train.ravel())

# Making predictions on the test data using the fitted model
y_pred = lin_reg.predict(X_test)

best_regressor["linear_regression"] = lin_reg

# Calculating mean squared error and r^2 score using hand-made functions
mse = MSE(y_test, y_pred)
r2 = R2(y_test, y_pred)

# Printing the results
print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

Mean Squared Error: 8.503855297262618
R^2 Score: 0.752405405556227


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV

svr_rbf = SVR()

param_grid = {
    "gamma": ["scale", "auto"],
    "C": np.random.uniform(low=0.001, high=100, size=10 ** 4),
    "epsilon": np.random.exponential(scale=0.1, size=10 ** 4),
    "shrinking": [True, False],
}

best_regressor["SVR"] = RandomizedSearchCV(
    estimator=svr_rbf,
    param_distributions=param_grid,
    n_iter=30,
    cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=-1,  # use all cores
    verbose=1,
    random_state=42,
)


In [ ]:
best_regressor["SVR"]

RandomizedSearchCV(cv=5, estimator=SVR(), n_iter=30, n_jobs=-1,
                   param_distributions={'C': array([74.75013344, 43.83510999, 55.12225998, ..., 21.46343913,
       68.42638294,  0.09515158]),
                                        'epsilon': array([0.12527286, 0.03198244, 0.11259649, ..., 0.0341358 , 0.28843126,
       0.34013291]),
                                        'gamma': ['scale', 'auto'],
                                        'shrinking': [True, False]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [ ]:
best_regressor["SVR"].fit(X_train, y_train.ravel())

y_pred = best_regressor["SVR"].predict(X_test)

# Calculating mean squared error and r^2 score using hand-made functions
mse = MSE(y_test, y_pred)
r2 = R2(y_test, y_pred)

# Printing the results
print("Mean Squared Error:", mse)
print("R^2 Score:", r2)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Mean Squared Error: 8.625387429101478
R^2 Score: 0.7488669282606155


In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor()  # rbf kernel by default
param_grid = {
    "max_depth": np.random.randint(low=1, high=10, size=100),
    "min_samples_split": np.random.randint(low=2, high=10, size=100),
    "min_samples_leaf": np.random.randint(low=1, high=10, size=100),
    "min_weight_fraction_leaf": np.random.exponential(scale=0.01, size=100),
}
best_regressor["decision_tree"] = RandomizedSearchCV(
    estimator=tree,
    param_distributions=param_grid,
    n_iter=100,
    cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=-1,  # use all cores
    verbose=1,
    random_state=42,
)
best_regressor["decision_tree"].fit(X_train, y_train.ravel())

y_pred = best_regressor["decision_tree"].predict(X_test)

# Calculating mean squared error and r^2 score using hand-made functions
mse = MSE(y_test, y_pred)
r2 = R2(y_test, y_pred)

# Printing the results
print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Mean Squared Error: 9.119800157929184
R^2 Score: 0.7344718198300499


In [ ]:
best_regressor

{'linear_regression': LinearRegression(),
 'SVR': RandomizedSearchCV(cv=5, estimator=SVR(), n_iter=30, n_jobs=-1,
                    param_distributions={'C': array([74.75013344, 43.83510999, 55.12225998, ..., 21.46343913,
        68.42638294,  0.09515158]),
                                         'epsilon': array([0.12527286, 0.03198244, 0.11259649, ..., 0.0341358 , 0.28843126,
        0.34013291]),
                                         'gamma': ['scale', 'auto'],
                                         'shrinking': [True, False]},
                    random_state=42, scoring='neg_mean_squared_error',
                    verbose=1),
 'decision_tree': RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_iter=100,
                    n_jobs=-1,
                    param_distributions={'max_depth': array([8, 2, 9, 3, 6, 8, 1, 4, 2, 7, 4, 5, 8, 9, 6, 3, 6, 2, 8, 1, 1, 1,
        5, 7, 7, 2, 8, 6, 5, 6, 1, 9, 1, 5, 5, 6, 8, 3, 5, 7, 4, 6, 9, 1,
        7, 9, 3, 4, 7, 8, 2, 1

$\mathbf{Exercise\, 3.}$ Plot the regression results (training and testing) for each model

In [ ]:
colnames = ["regressor", "type", "metric", "value"]
reg_results = []
types_data = [("train", X_train, y_train), ("test", X_test, y_test)]
metrics = [("MSE", mean_squared_error), ("r2_score", r2_score)]

for reg_name, reg in best_regressor.items():
    for type_name, X_type, y_type in types_data: 
        y_pred = reg.predict(X_type)
        for metric_name, metric_func in metrics:
            metric_val = metric_func(y_type, y_pred)
            reg_results.append([reg_name, type_name, metric_name, metric_val])

reg_results = pd.DataFrame(reg_results, columns=colnames)

In [ ]:
reg_results

,regressor,type,metric,value
0,linear_regression,train,MSE,8.343366
1,linear_regression,train,r2_score,0.747484
2,linear_regression,test,MSE,8.503855
3,linear_regression,test,r2_score,0.752405
4,SVR,train,MSE,8.340746
5,SVR,train,r2_score,0.747563
6,SVR,test,MSE,8.625387
7,SVR,test,r2_score,0.748867
8,decision_tree,train,MSE,7.803001
9,decision_tree,train,r2_score,0.763838


In [ ]:
fig = px.bar(
    reg_results,
    x="regressor",
    y="value",
    color="type",
    barmode="group",
    text_auto=True,
    facet_row="metric",
)

fig


We see that linear regression regression doesn't perform as well on the train set, but it outperforms its two rivals on the test set, at least in terms of both metrics we chose.

$\mathbf{Exercise\, 4.}$ Create a classification benchmark with 3 labels.

Evaluate training and testing prediction metrics for the following classification methods:

- K-nearest neighbours
- Random Forest
- Naive Bayes

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
from sklearn.datasets import make_classification

In [ ]:
X, y = make_classification(n_samples=1000, n_features=4, n_informative=4, n_redundant=0, n_classes=3, random_state=42)


data = pd.DataFrame(X, columns=["feature_" + str(i) for i in range(1, 5)])
data["label"] = y

data.head()

,feature_1,feature_2,feature_3,feature_4,label
0,-0.268999,2.047059,-4.687296,-0.873852,2
1,0.439745,-1.884729,0.264197,1.734492,1
2,0.002481,1.873506,0.160924,2.684806,2
3,-0.664447,-0.993198,-1.611478,1.376072,1
4,2.829484,2.541624,0.968068,-0.099387,0


In [ ]:
fig = px.histogram(
    data.melt("label"),
    x="value",
    color="label",
    facet_col="variable",
    opacity=0.7,
    barmode="overlay",
    color_discrete_sequence=px.colors.qualitative.Vivid,
    title="Distribution of labels",
)

fig

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((800, 4), (200, 4), (800,), (200,))

In [ ]:
best_classifier = {}

# KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier()
param_grid = {
    "n_neighbors": [3], 
    "leaf_size": np.random.randint(low=10, high=50, size=50),
    "weights": ["uniform", "distance"],
    "p": np.random.randint(low=1, high=5, size=5),
}

best_classifier["KNN"] = RandomizedSearchCV(
    estimator=knn,
    param_distributions=param_grid,
    n_iter=500,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,  # use all cores
    verbose=1,
    random_state=42,
)
best_classifier["KNN"].fit(X_train, y_train.ravel())

y_pred = best_classifier["KNN"].predict(X_test)

print("F1 score::", f1_score(y_test, y_pred, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
F1 score:: 0.8969042533374018
Accuracy: 0.895


# **Random Forest Classifier**

In [ ]:
rfc = RandomForestClassifier()
param_grid = {
    "n_estimators": np.random.randint(low=50, high=200, size=1000),
    "criterion": ["gini", "entropy"],
    "max_depth": np.random.randint(low=1, high=10, size=1000),
    "min_samples_split": np.random.randint(low=2, high=10, size=1000),
    "min_samples_leaf": np.random.randint(low=1, high=10, size=1000),
    "min_weight_fraction_leaf": np.random.exponential(scale=0.01, size=1000),
}

best_classifier["RFC"] = RandomizedSearchCV(
    estimator=rfc,
    param_distributions=param_grid,
    n_iter=100,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,  # use all cores
    verbose=1,
    random_state=42,
)

best_classifier["RFC"].fit(X_train, y_train.ravel())

y_pred = best_classifier["RFC"].predict(X_test)

print("F1 score::", f1_score(y_test, y_pred, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred))


Fitting 5 folds for each of 100 candidates, totalling 500 fits
F1 score:: 0.8501205366153249
Accuracy: 0.85


# Gausian NB

In [ ]:
gnb = GaussianNB()
param_grid = {}  

best_classifier["GNB"] = RandomizedSearchCV(
    estimator=gnb,
    param_distributions=param_grid,
    n_iter=1,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,  # use all cores
    verbose=1,
    random_state=42,
)
best_classifier["GNB"].fit(X_train, y_train.ravel())

y_pred = best_classifier["GNB"].predict(X_test)


print("F1 score::", f1_score(y_test, y_pred, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
F1 score:: 0.6854358846505443
Accuracy: 0.7


In [ ]:
colnames = ["classifier", "type", "metric", "value"]
results = pd.DataFrame(columns=colnames)
types_data = [["train", X_train, y_train], ["test", X_test, y_test]]
metrics = [["f1_score", f1_score], ["accuracy", accuracy_score]]

for clf_name, clf in best_classifier.items():
    for type_name, X_type, y_type in types_data: 
        y_pred = clf.predict(X_type)
        for metric_name, metric_func in metrics:
            if metric_func == f1_score:
                metric_val = metric_func(y_type, y_pred, average="macro")
            else:
                metric_val = metric_func(y_type, y_pred)
            
            row = pd.Series(
                dict(zip(colnames, [clf_name, type_name, metric_name, metric_val]))
            )
            results = results.append(row, ignore_index=True)

results = pd.DataFrame(results, columns=colnames)

In [ ]:
results

,classifier,type,metric,value
0,KNN,train,f1_score,0.912483
1,KNN,train,accuracy,0.912500
2,KNN,test,f1_score,0.896904
3,KNN,test,accuracy,0.895000
4,RFC,train,f1_score,0.931035
5,RFC,train,accuracy,0.931250
6,RFC,test,f1_score,0.850121
7,RFC,test,accuracy,0.850000
8,GNB,train,f1_score,0.645793
9,GNB,train,accuracy,0.653750


In [ ]:
fig = px.bar(
    data_frame=results,
    x="classifier",
    y="value",
    color="type",
    barmode="group",
    text_auto=True,
    facet_row="metric",
)

fig

We see that KNN performs the best. We could have explored several number of classes with hyperparameter-tuning and we would have found that there are three classes. We see that KNN outperforms RFC and GNB in terms of the f1-score and the accuracy on the test set.